# Получим датасет из файла

**Подключим библиотеки**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
RANDOM_STATE = 42

In [3]:
df = pd.read_pickle ("fix_data.pkl ")

In [4]:
df.head()

,Age,Customer Type,Type of Travel,Inflight wifi service,Ease of Online booking,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,satisfaction,Class_Eco,Class_Eco Plus
0,48.0,0,1,3.0,3.0,5.0,3.0,5.0,5.0,3.0,2.0,5.0,4.0,5.0,5.0,0,0.0,0.0
1,35.0,1,1,2.0,2.0,3.0,5.0,4.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,1,0.0,0.0
2,41.0,1,1,4.0,4.0,5.0,5.0,5.0,3.0,3.0,3.0,3.0,4.0,3.0,5.0,1,0.0,0.0
3,50.0,1,1,2.0,2.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0,3.0,5.0,4.0,1,0.0,0.0
4,49.0,1,1,3.0,3.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,3.0,3.0,5.0,1,0.0,0.0


# Метод опорных векторов

# Разобъем данные на тренировочные и тестовые

In [5]:
y = df['satisfaction']

X = df.drop(columns='satisfaction')

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train.shape, X_test.shape,y_train.shape,y_test.shape

((77550, 17), (25851, 17), (77550,), (25851,))

**Импорт библиотек**

In [21]:

from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm

# Воспользуемся скалером

In [11]:
ss = MinMaxScaler()
ss.fit(X_train)

X_train = pd.DataFrame(ss.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(ss.transform(X_test), columns=X_test.columns)

X_train.head()

,Age,Customer Type,Type of Travel,Inflight wifi service,Ease of Online booking,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Class_Eco,Class_Eco Plus
0,0.54,1.0,0.0,0.6,0.6,1.0,0.6,0.8,1.0,1.0,1.0,0.8,1.0,0.8,1.0,1.0,0.0
1,0.37,1.0,1.0,1.0,1.0,0.6,1.0,0.8,1.0,1.0,1.0,1.0,0.6,1.0,0.8,0.0,0.0
2,0.38,1.0,1.0,0.8,0.8,1.0,0.8,0.8,0.6,0.6,0.8,0.6,1.0,0.6,0.6,0.0,0.0
3,0.59,1.0,1.0,0.4,0.4,0.8,1.0,0.8,0.6,0.6,0.6,0.6,0.8,0.6,0.6,0.0,0.0
4,0.48,1.0,1.0,0.4,0.4,0.8,0.8,1.0,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.0,0.0


# SVM и оценка важности признаков

# SVC kernel='linear'

In [27]:
C = 1.0  # = self._alpha in our algorithm
model= svm.SVC(kernel='linear', C=C, probability=True)

model.fit(X_train, y_train)

pred = model.predict(X_test)

Веса модели

In [32]:
model.coef_, model.intercept_

(array([[-0.59473735,  1.29100581,  1.8708581 ,  1.58170983, -0.78362942,
         -0.08176729,  2.06094409,  0.07504832,  0.28893156,  0.9705667 ,
          0.74234207,  0.35422951,  0.92497657,  0.30823961,  0.71300771,
         -0.49651053, -0.58402087]]),
 array([-6.95256339]))

In [33]:
importances = pd.DataFrame({'weights': model.coef_[0], 'features': X_train.columns}).sort_values(by='weights')
importances.head(18)

,weights,features
4,-0.783629,Ease of Online booking
0,-0.594737,Age
16,-0.584021,Class_Eco Plus
15,-0.496511,Class_Eco
5,-0.081767,Food and drink
7,0.075048,Seat comfort
8,0.288932,Inflight entertainment
13,0.308240,Inflight service
11,0.354230,Baggage handling
14,0.713008,Cleanliness


In [34]:
accuracy_score(y_test, pred)

0.8751692391010019

In [35]:
recall_score(y_test, pred)

0.8275646185493247

In [36]:
precision_score(y_test, pred)

0.8768953752843063

In [37]:
probs = model.predict_proba(X_test)

probs[:5]

array([[0.59538025, 0.40461975],
       [0.71821775, 0.28178225],
       [0.90075642, 0.09924358],
       [0.02684552, 0.97315448],
       [0.22933488, 0.77066512]])

In [38]:
classes = probs[:,1] > 0.8

classes[:5]

array([False, False, False,  True, False])

In [39]:
y_actual = pd.Series (y_test, name='Actual')
y_predicted = pd.Series (classes, name='Predicted')

#create confusion matrix
print(pd.crosstab (y_test, classes))

col_0         False  True 
satisfaction              
0             14362    308
1              3798   7383


In [40]:
confusion_matrix(y_test, classes), precision_score(y_test, classes)

(array([[14362,   308],
        [ 3798,  7383]], dtype=int64),
 0.9599531920426473)

In [41]:
recall_score(y_test, classes)

0.6603166085323317

In [42]:
f1_score(y_test, classes)

0.7824289953370073

#  svm LinearSVC

In [103]:
model1 = svm.LinearSVC(C=C, max_iter=10000)

model1.fit(X_train, y_train)

pred = model1.predict(X_test)

c:\work\bootcamp_ml\env\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [104]:
model1.coef_, model1.intercept_

(array([[-0.26518449,  0.64525636,  0.90108758,  0.6889455 , -0.42978003,
         -0.04403782,  0.93682915,  0.09757487,  0.16763649,  0.4454549 ,
          0.3872281 ,  0.18882928,  0.46218212,  0.16522893,  0.32433389,
         -0.25542046, -0.28976802]]),
 array([-3.30704825]))

In [105]:
importances = pd.DataFrame({'weights': model1.coef_[0], 'features': X_train.columns}).sort_values(by='weights')
importances.head(18)

,weights,features
4,-0.429780,Ease of Online booking
16,-0.289768,Class_Eco Plus
0,-0.265184,Age
15,-0.255420,Class_Eco
5,-0.044038,Food and drink
7,0.097575,Seat comfort
13,0.165229,Inflight service
8,0.167636,Inflight entertainment
11,0.188829,Baggage handling
14,0.324334,Cleanliness


In [106]:
accuracy_score(y_test, pred)

0.8727708792696608

In [107]:
recall_score(y_test, pred)

0.8326625525444952

In [108]:
precision_score(y_test, pred)

0.8678225205070843

In [109]:
f1_score(y_test, pred)

0.8498790451412662

#  svm.SVC(kernel='rbf', gamma=0.7, C=C)

In [94]:
model2 = svm.SVC(kernel='rbf', gamma=0.7, C=C)

model2.fit(X_train, y_train)

pred = model2.predict(X_test)

In [96]:
accuracy_score(y_test, pred)

0.9518780704808325

In [97]:
recall_score(y_test, pred)

0.9312226097844558

In [98]:
precision_score(y_test, pred)

0.956369982547993

In [ ]:
f1_score(y_test, pred)

# svm SVC kernel='poly', degree=3, gamma='auto'

In [87]:
model3 = svm.SVC(kernel='poly', degree=3, gamma='auto', C=C, probability=False)

model3.fit(X_train, y_train)

pred = model3.predict(X_test)

In [89]:
accuracy_score(y_test, pred)

0.9162121387954044

In [90]:
recall_score(y_test, pred)

0.867900903318129

In [91]:
f1_score(y_test, pred)

0.8996013720218782

In [92]:
precision_score(y_test, pred)

0.9337053786202252

# Вывод

модель svm.SVC(kernel='rbf', gamma=0.7, C=C) показала отличный результат определения целевой переменной, но так как этот тип моделей можно отнести к моделям типа "черного ящика", то его мы сможем использовать только как дополнительную характеристику.

# Сохранение модели

In [ ]:
import pickle

with open('model_svm.pickle', 'wb') as f:
    pickle.dump(model2, f)

